In [7]:
# Install necessary packages
%pip install nltk pandas matplotlib seaborn numpy scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.6 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 6.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import re
import nltkfrom nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/codespace/nltk_data...


True

In [13]:
requests = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/SYRCityline_Requests_(2021-Present).csv')
weather = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/40_Year_Temperature_Dataset_SU-2.csv')
ratings_2021 = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/Pavement_Ratings_(2021)-2.csv')
ratings_2022 = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/Syracuse_Pavement_Ratings_(2022).csv')
ratings_2023 = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/Syracuse_Ratings_Open_Data_Copy_2023_8469753568545483898.csv')
streets = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/City_Streets_2011.csv')

Approach:
1) Load the request data, filter out the pothole requests from 2021-2023
2) Standardize the address in potholes and match with the street dictionary 
3) Concatenate the rating data
4) Filter out temperature data
5) Merge  

In [14]:
requests['Description'] = requests['Description'].astype(str)
potholes = requests[requests['Category'] == 'Potholes']
# Ensure the 'Date' column is in datetime format
potholes = potholes[['Rating', 'Address', 'Description', 'Created_at_local', 'Minutes_to_closed']]
potholes['Date'] = pd.to_datetime(potholes['Created_at_local'], format='%m/%d/%Y - %I:%M%p').dt.date
# Define the date range
start_date = pd.to_datetime("2021-01-01").date()
end_date = pd.to_datetime("2023-12-31").date()
# Filter the DataFrame to the specified range
filtered_potholes = potholes[(potholes['Date'] >= start_date) & (potholes['Date'] <= end_date)]

Clean up the street names column

In [ ]:
# Function to extract cleaner street names
def extract_clean_street_name(address):
    # More strict extraction to focus on just the name and type
    match = re.search(r'\b([A-Za-z]+)\s*(St|Street|Ave|Avenue|Blvd|Boulevard|Dr|Drive|Rd|Road|Ln|Lane|Way|Circle|Cir)\b', address, re.IGNORECASE)
    if match:
        # Normalize the street name by removing common suffixes
        street_name = match.group(1).strip().upper()  # Extract and normalize the base name
        return street_name
    else:
        return None

# Apply the revised function to the Address column
filtered_potholes['Street'] = filtered_potholes['Address'].apply(extract_clean_street_name)
filtered_potholes['Street'] = filtered_potholes['Street'].astype(str)

/tmp/ipykernel_12337/577753340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_potholes['Street'] = filtered_potholes['Address'].apply(extract_clean_street_name)
/tmp/ipykernel_12337/577753340.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_potholes['Street'] = filtered_potholes['Street'].astype(str)


Caclulate sentiment analysis of potholes in relation to the street names

In [24]:
sia = SentimentIntensityAnalyzer()

def sentiment_score(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    
    # Join the tokens back into a string (optional step, depending on your approach)
    text = ' '.join(tokens)
    
    # Calculate sentiment score
    score = sia.polarity_scores(text)['compound']
    
    # Return the compound sentiment score
    return score

filtered_potholes['SentimentScore'] = filtered_potholes['Description'].apply(sentiment_score)

def categorize_severity(score):
    if score <= -0.5:
        return 'high'
    elif score <= 0:
        return 'medium'
    else:
        return 'low'

filtered_potholes['SeverityLevel'] = filtered_potholes['SentimentScore'].apply(categorize_severity)

potholes_agg = filtered_potholes.groupby(['Date', 'Street']).agg({
    'SentimentScore': 'mean',  # Average sentiment score per day
    'SeverityLevel': lambda x: x.mode()[0] if not x.empty else None,  # Most common severity level per day
    'Description': 'count',  # Count the number of reports per day
    'Rating': 'mean',  # Average rating
    'Minutes_to_closed': 'mean'  # Average minutes to close
}).rename(columns={'Description': 'Potholes_Count'}).reset_index()

# potholes_agg['Date'] = pd.to_datetime(potholes_agg['Date'])
potholes_agg.head()

NameError: name 'word_tokenize' is not defined